In [1]:
# Imports
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt 
# %matplotlib inline
from matplotlib.pylab import *
from matplotlib import gridspec
import copy
import scipy
from scipy.io                     import wavfile
from scipy                        import stats, signal
from scipy.fftpack                import fft
from scipy.signal                 import lfilter, hamming
from scipy.fftpack.realtransforms import dct
import threading
from scipy.signal import *
from pylab import *
from scipy.spatial.distance import euclidean
import math
# import arff
from sklearn import metrics, svm
import scipy.interpolate
from sklearn.metrics import auc
import numpy as np
import time
import datetime
from datetime import datetime
import csv
# from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
%autosave 15

Autosaving every 15 seconds


# Main

In [9]:
# Folder selection
# folderPath = '/Volumes/HanSolo/Dropbox/Georgia Tech/TipTap/Data/RingWatch/2017_01_24/p2/'
# folderPath = "/Users/gareyes/Downloads/Synchro/swipedata/"
# folderPath = "/Users/gareyes/Downloads/Pilot/P2/swipe/t1490397783698_p2_sit/"
folderPath = "/Users/jwpilly/Downloads/Study_v2/P2/swipe/t20170401141910_p2_sit/"
# folderPath = '/Users/jwpilly/Research/RingWatch/2017_01_23/'
dataPath = folderPath + "data/"
if not os.path.exists(dataPath):
    os.makedirs(dataPath)
figurePath = folderPath + "figures/"
if not os.path.exists(figurePath):
    os.makedirs(figurePath)

totalTimes = []
# Walk the datapath
for root, folders, files in os.walk(dataPath):
    for f in files:
        if "Store" not in f:
            
            # parse filename        
            filePath = dataPath + f
            figureFileName = f[:-4]
            figureFilePath = figurePath + figureFileName

            if "swipe" in f:
                
                print f
                
                # Load data
                swipeData = pd.read_csv(filePath, sep=',', header=None, low_memory=False)
                swipeData.columns = ['tsystem', 'action', 'x', 'y']

                # Total Length
                totalTime = calculateTotalTime(swipeData)
                totalTimes.append(totalTime)
                print totalTime

                #todo: figure out time from buzz to swipedetected
                #todo: figure out time from to lift to touchdown
                #todo: figure out time from to touchdown to touchup
                #todo: figure out time from touchup to swipedetected

                # Normalize time data
#                 swipeData = normalizeSystemTime(swipeData)

                # Plot
                startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
                downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
                endTime = swipeData.tsystem[swipeData[(swipeData.action == "swipe left") | (swipeData.action == "end") | (swipeData.action== "swipe right")].index.tolist()[0]]    
                fig = plt.figure()
#                 fig.set_size_inches(40,40)
#                 plt.xlim(-100,4000)
                plt.axvline(x=startTime)
                plt.axvline(x=downTime)
                plt.axvline(x=endTime)
                plt.savefig(figureFilePath + "_touch.png")
                plt.close()

            elif "wrist" in f:
            
                
                # Load data
                wristData = pd.read_csv(filePath, sep=',', header=None, low_memory=False)
                wristData.columns = ['sensor', 'index', 'tevent', 'tsystem', 'x', 'y', 'z']

                # Total Length
#                 print calculateTotalTime(wristData)

                # Normalize time data
#                 wristData = normalizeSystemTime(wristData)

                # Plot gyro
                fig = plt.figure()
                fig.set_size_inches(20,20)
                plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].x)
                plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].y)
                plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].z)
                startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
                downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
                endTime = swipeData.tsystem[swipeData[(swipeData.action == "swipe left") | (swipeData.action == "end") | (swipeData.action== "swipe right")].index.tolist()[0]]    
                # plt.xlim(-100,4000)
                plt.axvline(x=startTime)
                plt.axvline(x=downTime)
                plt.axvline(x=endTime)
                plt.savefig(figureFilePath + "_accel.png")
                plt.close()

                fig = plt.figure()
                fig.set_size_inches(20,20)
                plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].x)
                plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].y)
                plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].z)
                startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
                downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
                endTime = swipeData.tsystem[swipeData[(swipeData.action == "swipe left") | (swipeData.action == "end") | (swipeData.action== "swipe right")].index.tolist()[0]]    
                # plt.xlim(-100,4000)
                plt.axvline(x=startTime)
                plt.axvline(x=downTime)
                plt.axvline(x=endTime)
                plt.savefig(figureFilePath + "_gyro.png")
                plt.close()
                           
    print "mean: " + str(np.mean(totalTimes)/1000) + "s" + ", stdev: " + str(np.std(totalTimes)/1000) + "s"

mean: nans, stdev: nans


In [3]:
totalTimes

[]

In [ ]:
plt.plot(totalTimes)
# plt.ylim(0,3000)
plt.show()

# Helpers

In [4]:
def calculateTotalTime(df):
    startTime = df.tsystem[0]
    endTime = df.tsystem[len(df.tsystem)-1]
    return endTime - startTime

In [5]:
def normalizeEventTime(df):
    newDf = df
    newDf.tevent = newDf.tevent - newDf.tevent[0]
    return newDf

In [6]:
def normalizeSystemTime(df):
    newDf = df
    newDf.tsystem = newDf.tsystem - newDf.tsystem[0]
    return newDf

# Testing

In [7]:
if "vibrate" in swipeData.action:
    print swipeData.tevent

NameError: name 'swipeData' is not defined

In [ ]:
if 'vibrate' in swipeData.action.values:
    s
    

In [8]:
swipeData.action.index.tolist()


NameError: name 'swipeData' is not defined

In [ ]:
swipeData[swipeData.action == "vibrate"].index.tolist()

In [ ]:
swipeData[swipeData.action == "end"].index.tolist()[0]

In [ ]:
print swipeData.action[14]

In [ ]:
print swipeData.tevent[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]

In [ ]:
print normalizeTime(swipeData)

In [ ]:
df=swipeData
df.tevent = df.tevent - df.tevent[0]
print df


In [ ]:
fig = plt.figure()
fig.set_size_inches(40,40)
plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].x)
plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].y)
plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].z)
startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
endTime = swipeData.tsystem[swipeData[swipeData.action == "end"].index.tolist()[0]]    
# plt.xlim(-100,4000)
plt.axvline(x=startTime)
plt.axvline(x=downTime)
plt.axvline(x=endTime)
plt.show()

fig = plt.figure()
fig.set_size_inches(40,40)
plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].x)
plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].y)
plt.plot(wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].tsystem, wristData.loc[wristData['sensor'] == "android.sensor.gyroscope"].z)
startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
endTime = swipeData.tsystem[swipeData[swipeData.action == "end"].index.tolist()[0]]    
# plt.xlim(-100,4000)
plt.axvline(x=startTime)
plt.axvline(x=downTime)
plt.axvline(x=endTime)
plt.show()


In [ ]:
startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
endTime = swipeData.tsystem[swipeData[swipeData.action == "end"].index.tolist()[0]]    
fig = plt.figure()
# plt.xlim(-100,4000)
plt.axvline(x=startTime)
plt.axvline(x=downTime)
plt.axvline(x=endTime)
plt.show()


In [ ]:
"gyroscope" in wristData.sensor

In [ ]:
print wristData.sensor

In [ ]:
wristData.loc[wristData['sensor'] == "android.sensor.accelerometer"].y

In [10]:
def processSwipeFile(folderPath):    
    dataPath = folderPath + "data/"
    if not os.path.exists(dataPath):
        os.makedirs(dataPath)

    totalTimes = []
    # Walk the datapath
    for root, folders, files in os.walk(dataPath):
        for f in files:
            if "Store" not in f:

                # parse filename        
                filePath = dataPath + f

                if "swipe" in f:

                    # print f

                    # Load data
                    swipeData = pd.read_csv(filePath, sep=',', header=None, low_memory=False)
                    swipeData.columns = ['tsystem', 'action', 'x', 'y']

                    # Total Length
                    totalTime = calculateTotalTime(swipeData)
                    totalTimes.append(totalTime)
                    # print totalTime

                    #todo: figure out time from buzz to swipedetected
                    #todo: figure out time from to lift to touchdown
                    #todo: figure out time from to touchdown to touchup
                    #todo: figure out time from touchup to swipedetected

                    # Normalize time data
    #                 swipeData = normalizeSystemTime(swipeData)

                    # Plot
                    startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
                    downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
                    endTime = swipeData.tsystem[swipeData[(swipeData.action == "swipe left") | (swipeData.action == "end") | (swipeData.action== "swipe right")].index.tolist()[0]]    
                    fig = plt.figure()
    #                 fig.set_size_inches(40,40)
    #                 plt.xlim(-100,4000)

                elif "wrist" in f:


                    # Load data
                    wristData = pd.read_csv(filePath, sep=',', header=None, low_memory=False)
                    wristData.columns = ['sensor', 'index', 'tevent', 'tsystem', 'x', 'y', 'z']

                    # Total Length
    #                 print calculateTotalTime(wristData)

                    # Normalize time data
    #                 wristData = normalizeSystemTime(wristData
                    startTime = swipeData.tsystem[swipeData[swipeData.action == "vibrate"].index.tolist()[0]]
                    downTime = swipeData.tsystem[swipeData[swipeData.action == "touch_down"].index.tolist()[0]]
                    endTime = swipeData.tsystem[swipeData[(swipeData.action == "swipe left") | (swipeData.action == "end") | (swipeData.action== "swipe right")].index.tolist()[0]]    

        # print "mean: " + str(np.mean(totalTimes)/1000) + "s" + ", stdev: " + str(np.std(totalTimes)/1000) + "s"
        return np.mean(totalTimes) / 1000

In [11]:
print(processSwipeFile("/Users/jwpilly/Downloads/Study_v2/P2/swipe/t20170401141910_p2_sit/"))

nan


In [ ]:
from functools import reduce

baseFilePath = "/Users/jwpilly/Documents/Research/Synchro/Study_v2/"
presults = []
for root, folders, files in os.walk(baseFilePath):
    if not root.endswith("swipe"):
        continue
    sfolders = sorted(folders, key=lambda x : int(x.split("_")[0].replace("t", "")))
    efolders = [f for f in sfolders[:] if "prep" not in f]
    efolderpaths = [root + "/" + f + "/" for f in efolders]
    efoldertimes = [(efolderpaths[i].split("_")[-1][:-1] + str(i), processSwipeFile(efolderpaths[i])) for i in range(len(efolderpaths))]
    presults.append(efoldertimes)
presultsa = [[i[1] for i in t] for t in [sorted(r) for r in presults]]
paverages = reduce(lambda x, y : np.array(x) + np.array(y), presultsa) / len(presultsa)
print(paverages)

In [ ]:
%matplotlib inline
plt.clf()
fig = plt.figure(figsize=(6, 4))
objects = ('Practice 1', 'Practice 2', 'Eval 1', 'Eval 2')
y_pos = np.arange(len(objects))
performance = paverages
# plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.plot(y_pos, [paverages[i] for i in range(4)], label="Sit")
plt.plot(y_pos, [paverages[i + 4] for i in range(4)], label="Walk")
plt.xticks(y_pos, objects)
plt.ylim((1.5,2.5))
plt.ylabel('Time (s)')
plt.title('Swipe Times')
plt.legend(loc="best")
plt.savefig("swipetimes.pdf")